# EmailQExpress Version 2 (HTML Email Body Content)

In [ ]:
import win32com.client
import os
import base64
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from datetime import datetime, timedelta

# Email configuration
email_address = 'Input your email address'
password = 'Input your password'

# Connect to Outlook
outlook_app = win32com.client.Dispatch("Outlook.Application")
namespace = outlook_app.GetNamespace("MAPI")
inbox = namespace.GetDefaultFolder(6)  # Inbox folder index

# Get the current date and calculate the date for two months ago
# current_date = datetime.now()
# two_months_ago = current_date - timedelta(days=current_date.day)
# previous_month_date = two_months_ago - timedelta(days=two_months_ago.day)
# previous_month_year = previous_month_date.strftime('%B %Y')

# Get the current date and calculate the previous month's date
current_date = datetime.now()
previous_month_date = current_date - timedelta(days=current_date.day)
previous_month_year = previous_month_date.strftime('%B %Y')

# Path to the Un-updated and Rejected Records Excel files
excel_files_path = r"Put the filepath to 'Un-updated and Rejected Files'"

# Get a list of all subfolders in the excel files path
subfolders = [f.path for f in os.scandir(excel_files_path) if f.is_dir()]

# Path to the Contact Details Excel file
contact_details_path = r"Put the filepath to 'Contact Details.xlsx'"

# Define the yellow fill
yellow_fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")

# Path to the image file
image_file_path = r"Put the filepath to 'Signature.jpg'"

# Read the image file as binary data
with open(image_file_path, "rb") as image_file:
    # Encode the binary data into Base64 string
    encoded_image = base64.b64encode(image_file.read()).decode("utf-8")

print("Processing Draft Emails...")
print('#'*100)

# Function to perform data transformation on "Rejected Accounts" files
def transform_rejected_accounts(sheet):
    for row in sheet.iter_rows(min_row=2, min_col=2, max_col=2):
        for cell in row:
            value = cell.value
            if value:
                if value.startswith("Structure Reject::Ownership Type="):
                    cell.value = ""
                elif value.startswith("Record ignored as the related parent / mandatory segment"):
                    cell.value = ""
                elif value.startswith("Principal Amount Overdue was not reported"):
                    cell.value = "[In 'Credit Information', no corresponding 'Days in Arrears' was reported when 'Overdue Amount' was greater than zero.]"
                elif value.startswith("Number of Days Past Due was not reported"):
                    cell.value = "[In 'Credit Information', no corresponding 'Overdue Amount' was reported when 'Days in Arrears' was greater than zero.]"
                elif value.startswith("Microfinance Consumer Subject Segment.Date of Birth/ Registration Date was not reported"):
                    cell.value = "The 'Date of Birth' was not or wrongly reported."

# Create and save multiple draft emails
for subfolder_path in subfolders:
    subfolder_name = os.path.basename(subfolder_path)
    subject = f"DATA SUBMISSION FEEDBACK FOR {previous_month_year.upper()} - {subfolder_name}"
                                
    # Create a draft email
    html_body = f"""
    <html>
      <body style="font-family: 'Futura Bk Bt', sans-serif; font-size: 16px;">
        <br>
        <br>
        <p>Good day,</p>
        <br>
        <p>I trust this meets you well.</p>
        <br>
        <p>Kindly be informed that your {previous_month_year} credit data has been processed successfully. However, during our review process, we set aside some records for your correction. These include, attached if applicable;</p>
        <p>1. Subjects with no identifiers (attached if any) due to the bureau’s communication with your bank.</p>
        <p>2. Facility records with no subject information (attached if any).</p>
        <br>
        <p>Below is a summary of your institution’s data quality.</p>
        <table border="1" cellpadding="5" style="border-collapse: collapse;">
          <tr>
            <th style="border: 1px solid black; text-align: left;">{subfolder_name.upper()}<br>{previous_month_year.upper()}</th>
            <th style="border: 1px solid black;"> TOTAL RECORD LOADED </th>
            <th style="border: 1px solid black;"> TOTAL RECORD PROCESSED </th>
            <th style="border: 1px solid black;"> TOTAL RECORD REJECTED </th>
            <th style="border: 1px solid black;"> PERCENTAGE PROCESSED </th>
          </tr>
          <tr>
            <td style="border: 1px solid black;"> CONSUMER</td>
            <td style="border: 1px solid black; text-align: center;">CONSLOADED</td>
            <td style="border: 1px solid black; text-align: center;">CONSPROCESSED</td>
            <td style="border: 1px solid black; text-align: center;">CONSREJECTED</td>
            <td style="border: 1px solid black; text-align: center;">CONS%</td>
          </tr>
          <tr>
            <td style="border: 1px solid black;"> CORPORATE</td>
            <td style="border: 1px solid black; text-align: center;">COMMLOADED</td>
            <td style="border: 1px solid black; text-align: center;">COMMPROCESSED</td>
            <td style="border: 1px solid black; text-align: center;">COMMREJECTED</td>
            <td style="border: 1px solid black; text-align: center;">COMM%</td>
          </tr>
        </table>
        <br>
        <!--<p>Please find attached <strong>un-updated </strong> and <strong>rejected accounts</strong> for your review.</p>-->
        <p>Please find attached <strong>un-updated</strong> and <strong>rejected accounts</strong> for your review.</p>
        <p>NB: The columns in colour in the <strong>un-updated accounts</strong> are to be populated with the accounts' current position and reverted for processing to maintain accurate/updated records of your customers.</p>
        <br>
        <p><strong>More so, always endeavour to provide valid BVNs and RC Numbers for all individuals and corporate entities in your subsequent submissions.</p></strong>
        <br>
        <p>Should you require any clarification please don't hesitate to contact us.</p>
        <br>
        <p>Kind regards,</p>
        <img src="data:image/png;base64,{encoded_image}" alt="Signature" width="446in" height="212in">
        <p style="font-size: 15px;">Please click on the links below to follow us on social media:</p>
        <p style="font-size: 15px;">Twitter: @CRCCreditbureau Facebook: @CRCCreditBureau Instagram: @crccreditbureau</p>
        <p style="font-size: 15px;">LinkedIn: https://www.linkedin.com/company/credit-reference-company/</p>
      </body>
    </html>
    """

    new_mail = outlook_app.CreateItem(0)  # 0 represents MailItem
    new_mail.Subject = subject
    new_mail.HTMLBody = html_body
    
    # Attach Excel files from corresponding subfolder
    unupdated_excel_files = [
        f.path for f in os.scandir(subfolder_path) 
        if f.name.endswith('.xlsx') and f.name.startswith("Un-Updated Accounts")
    ]

    for excel_file in unupdated_excel_files:
        if os.path.exists(excel_file):
            # Load the workbook
            wb = load_workbook(excel_file)
            for ws in wb:
                # Apply yellow fill to columns E to I in all worksheets
                for row in ws.iter_rows(min_row=2, min_col=5, max_col=9):
                    for cell in row:
                        cell.fill = yellow_fill

            # Save the workbook with the formatting
            wb.save(excel_file)

            # Attach the formatted Excel file
            attachment = new_mail.Attachments.Add(excel_file)
        else:
            print(f"Attachment {excel_file} not found. Skipping...")
    
    
    rejected_excel_files = [
        f.path for f in os.scandir(subfolder_path) 
        if f.name.endswith('.xlsx') and f.name.startswith("Rejected Accounts")
    ]

    for excel_file in rejected_excel_files:
        if os.path.exists(excel_file):
            # Load the workbook
            wb = load_workbook(excel_file)
            for ws in wb:
                # Perform data transformation on 'Rejected Accounts' files
                if ws.title.startswith('Rejected Accounts'):
                    transform_rejected_accounts(ws)

            # Save the transformed workbook
            wb.save(excel_file)

            # Attach the transformed Excel file
            attachment = new_mail.Attachments.Add(excel_file)
        else:
            print(f"Attachment {excel_file} not found. Skipping...")
    
    
    # List all Excel files in the subfolder except those starting with "Un-Updated & Rejected Accounts"
    other_excel_files = [
        f.path for f in os.scandir(subfolder_path) 
        if f.name.endswith('.xlsx') and not (f.name.startswith("Un-Updated Accounts") or f.name.startswith("Rejected Accounts"))
    ]

    for excel_file in other_excel_files:
        if os.path.exists(excel_file):
            # Load the workbook
            wb = load_workbook(excel_file)

            # Save the workbook with the formatting (if any changes were made)
            wb.save(excel_file)

            # Attach the formatted Excel file
            attachment = new_mail.Attachments.Add(excel_file)
        else:
            print(f"Attachment {excel_file} not found. Skipping...")

    
    pdf_files = [
        f.path for f in os.scandir(subfolder_path) 
        if f.name.endswith(('.pdf','.docx','.csv','.xls','.xlm','.xlb'))
    ]
    for pdf_file in pdf_files:
        if os.path.exists(pdf_file):
   
            attachment = new_mail.Attachments.Add(pdf_file)
    
    
    # Read recipients' email addresses from Contact Details Excel file
    wb = load_workbook(contact_details_path)
    ws = wb.active
    pry_email_addresses = []
    sec_email_addresses = []
    for row in ws.iter_rows(min_row=2, values_only=True):
        if row[1] == subfolder_name:  # Assuming INSTITUTION_NAME column is the second column (index 1)
            primary_recipient = row[3]  # Assuming PRIMARY_RECIPIENT column is the fourth column (index 3)
            other_recipients = row[4]  # Assuming OTHER_RECIPIENTS column is the fifth column (index 4)
            if primary_recipient:
                pry_email_addresses.extend(primary_recipient.split(';'))        
            if other_recipients:
                sec_email_addresses.extend(other_recipients.split(';'))
    
    # Add recipients to To: field
    if pry_email_addresses:
        new_mail.To = ';'.join(pry_email_addresses)
        new_mail.Recipients.ResolveAll()  # Resolve to be compatible with Outlook 'To: recipients' field
    # Add recipients to Cc: field
    if sec_email_addresses:
        new_mail.CC = ';'.join(sec_email_addresses)
        new_mail.Recipients.ResolveAll()  # Resolve to be compatible with Outlook 'CC: recipients' field
    
    new_mail.Save()

    print(f"Draft Email for {subfolder_name} saved successfully!")

print('#'*100)
print("All drafts saved successfully!")
